This notebook is for further investigation of the Potts model in predicting protein fitness with MSA. 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from model import CouplingsModel
import tools
import scipy
from pathlib import Path
from collections import OrderedDict
# biopython SeqIO
from Bio import SeqIO
from sklearn.metrics import roc_auc_score
import scipy

In [5]:
NAME = 'PABP_YEAST'

### training code for model, uncomment the following line: 
#!plmc/bin/plmc -o PABP_YEAST/model/{NAME}.model_params -c PABP_YEAST/model/{NAME}.txt -f PABP_YEAST -le 16.2 -lh 0.01 -m 200 -t 0.2 -g PABP_YEAST/data/{NAME}.a2m

In [18]:
# load parameters from file to create a pairwise model
c = CouplingsModel(f"PABP_YEAST/model/{NAME}.model_params")
# read the experimental mutational scanning dataset for PABP by Melamed et al., RNA, 2013
data = pd.read_csv(
    "PABP_YEAST/data/PABP_YEAST_Fields2013-singles.csv", sep=";", comment="#"
)
wt = {"mutant": "wild", "linear": 0, "log": 0}
data = data.append(wt, ignore_index=True)

/tmp/ipykernel_1425607/1563486686.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(wt, ignore_index=True)


In [21]:

# predict mutations using our model
data_pred = tools.predict_mutation_table(
    c, data, "effect_prediction_epistatic"
)

# can also add predictions by the corresponding independent model
c0 = c.to_independent_model()

data_pred = tools.predict_mutation_table(
    c0, data_pred, "effect_prediction_independent"
)
print(data_pred.head())
pred = data_pred['effect_prediction_epistatic']
pred_ind = data_pred['effect_prediction_independent']
label = data_pred['linear']
print('Evmutation spearman: ', scipy.stats.spearmanr(pred, label))
print('Independent control spearman: ', scipy.stats.spearmanr(pred_ind, label))

  mutant    linear       log  effect_prediction_epistatic  \
0  G126A  0.711743 -0.490571                    -2.601740   
1  G126C  0.449027 -1.155127                    -5.648695   
2  G126E  0.588928 -0.763836                    -6.598581   
3  G126D  0.229853 -2.121218                    -7.250636   
4  G126N  0.679435 -0.557593                    -5.800188   

   effect_prediction_independent  
0                       0.406487  
1                      -0.027602  
2                      -1.827570  
3                      -1.180076  
4                       0.387440  
Evmutation spearman:  SpearmanrResult(correlation=0.5887727501243497, pvalue=7.616520847741117e-112)
Independent control spearman:  SpearmanrResult(correlation=0.420793816603478, pvalue=3.2177604726528132e-52)


In [24]:
yeast_seq_str = []
fasta_sequences = SeqIO.parse(open("PABP_YEAST/data/PABP_YEAST.a2m"),'fasta')
for fasta in fasta_sequences:
    yeast_seq_str.append(str(fasta.seq))

In [38]:
pos_upper = [x for x in range(len(yeast_seq_str[0])) if not yeast_seq_str[0][x].islower()]
processed_yeast = np.asarray([np.asarray(list(s))[pos_upper] for s in yeast_seq_str])
processed_yeast.shape
wildtype = yeast_seq_str[0]

In [46]:
mutant, label = data['mutant'].to_numpy(), data['linear'].to_numpy()
mutant_data = []
for m in mutant:
    if m == 'wild':
        mutant_data.append(np.asarray(list(wildtype))[pos_upper])
        continue
    original_aa, loc, mutant_aa = m[0], int(m[1:4])-115, m[4]
    assert wildtype[loc] == original_aa
    mut_seq = np.asarray(list(wildtype[:loc]+mutant_aa+wildtype[loc+1:]))
    mutant_data.append(mut_seq[pos_upper])
mutant_data = np.asarray(mutant_data)
mutant_data

array([['K', 'G', 'S', ..., 'L', 'S', 'R'],
       ['K', 'G', 'S', ..., 'L', 'S', 'R'],
       ['K', 'G', 'S', ..., 'L', 'S', 'R'],
       ...,
       ['K', 'G', 'S', ..., 'L', 'S', 'R'],
       ['K', 'G', 'S', ..., 'L', 'S', 'R'],
       ['K', 'G', 'S', ..., 'L', 'S', 'R']], dtype='<U1')

In [48]:
# c.hamiltonians(mutant_data)
c.delta_hamiltonian

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(float32, 2d, C), Tuple(int64, [unichr x 1]))
 
There are 22 candidate implementations:
   - Of which 20 did not match due to:
   Overload of function 'getitem': File: <numerous>: Line N/A.
     With argument(s): '(array(float32, 2d, C), Tuple(int64, [unichr x 1]))':
    No match.
   - Of which 2 did not match due to:
   Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 166.
     With argument(s): '(array(float32, 2d, C), Tuple(int64, [unichr x 1]))':
    Rejected as the implementation raised a specific error:
      NumbaTypeError: unsupported array index type [unichr x 1] in Tuple(int64, [unichr x 1])
  raised from /data/anaconda3/envs/dev_arthur/lib/python3.9/site-packages/numba/core/typing/arraydecl.py:72

During: typing of intrinsic-call at /home/arthur/dev/msa_clas

In [ ]:
NAME = 'PABP_YEAST_10000'
c = CouplingsModel(f"PABP_YEAST/model/{NAME}.model_params")
pos_upper = [x for x in range(len(yeast_seq_str[0])) if not yeast_seq_str[0][x].islower()]
processed_yeast = [s for s in yeast_seq_str if not 'X' in s]
processed_yeast = [np.asarray(list(s))[pos_upper] for s in processed_yeast]
processed_yeast = [s for s in processed_yeast if not '-' in s]
# np.asarray(list(s))[pos_upper] for s in yeast_seq_str if not '-' in np.asarray(list(s))[pos_upper] and not 'X' in np.asarray(list(s))[pos_upper]
energies = c.hamiltonians(processed_yeast)[:, 0]
wt_energy = energies[0]

# c.hamiltonians([s])
data = pd.read_csv(
    "PABP_YEAST/data/PABP_YEAST_Fields2013-singles.csv", sep=";", comment="#"
)

# predict mutations using our model
data_pred = tools.predict_mutation_table(
    c, data, "effect_prediction_epistatic"
)
evm_pred = data_pred['effect_prediction_epistatic'].to_numpy()
evm_pred_pos = evm_pred[np.where(label>=1)]+wt_energy
evm_pred_neg = evm_pred[np.where(label<1)]+wt_energy